![overview](resources/images/IOTGW-OVERVIEW-HIGHLIGHT-GW.png)



### IIOT Gateway
---
- The IIOT gateway in our implementation is a Raspberry Pi-3(RPi3) which is a small Single Board Computer (SBC).
- The RPi3 runs a Debian OS which is similar to the Ubuntu/Linux OS we use in our desktop.
- Many softwares are run on the RPi3 which enable the RPi3 to provide the necessary services needed to be present on a IIOT Gateway.

#### Services Running on the RPi3
---
1. IIOT End Device Driver (IED Driver)
2. Mosquitto MQTT Broker
3. MQTT to IIOT Middleware Publish Service. 

#### The diagram below shows the microservices running on the RPi3 -
---
![IIOT Gateway Architecture](resources/images/IOTGW-DETAILS.jpg)


---
#### IIOT End Device Driver
---

![IIOT Gateway IED Driver](resources/images/IOTGW-DETAILS-HIGHLIGHT-IEDDRIVER.png)

- Now we will develop the Device Driver for our Arduino IIOT End device
- We will use Python to develop the device driver.
- Since we are using Serial UART over the USB, we will need the **pySerial** python library to read the data coming from the Arduino Serial Port.
- Run the code below to install the pySerial python package.

In [ ]:
pip install pyserial

---
##### Now we will list the available Serial ports connected to this machine already.
##### Do not connect the Arduino yet (Remove if already connected)
##### Run the snippet below.

##### It is fine if no serial ports are detected

---

In [ ]:
import serial.tools.list_ports

def detectSerialPorts():
    ports=list(serial.tools.list_ports.comports())
    if len(ports):
        print("Listing USB Serial ports -")
    else:
        print("No USB Serial Ports found.")
    for p in ports:
        print ("-",p.device)
    return map(lambda x: x.device ,ports)
        
ports=detectSerialPorts()

---
- Depending on your OS, the ports maybe COMx (for Windows) or /dev/ttyACMx and /dev/ttyUSBx (for Linux).
######  Now plug in the Arduino in which we already uploaded the Analog Read code and call the detectSerialPorts() function again to know the new device added.  
---

In [ ]:
updatedPorts=detectSerialPorts()
newPorts = list(set(updatedPorts) - set(ports))

if len(newPorts)!=0:
    arduinoPort = newPorts[0]
    print ("\nDetected new serial port -\n- {}".format(arduinoPort))
else:
    print("\nNo new serial port detected !!")


---
###### Now we can read the incoming analog values from the Arduino end device. Run the following snippet.
---

In [ ]:
import serial
s=serial.Serial(arduinoPort,baudrate=9600,timeout=1)
def readAnalog():
    val = s.readline()
    if val:
        try:
            data=int(val.strip().decode())
            return data
        except Exception as e:
            print(e)
            print("Try reading again.")
    else:
        print("Couldn't read analog value from Arduino IED.")

readAnalog()

---
###### We can read the Analog values by calling readAnalog. Run the code below.
---

In [ ]:
readAnalog()

#### Exercise T02A

- Complete the function below to read the sensor values of the Arduino IED and find the mean and standard deviation of the last N samples and return then in a tuple.


In [ ]:
import numpy as np
def readAnalogStats(N):
    values=[]
    for i in range(N):
        values.append(readAnalog()) # complete this line
    
    
    npvalues = np.array(values)
    return npvalues.mean(),npvalues.std()
        
# Call the function
readAnalogStats(100)

---
#### At this point, we have successfully written a simple device driver for our Arduino IED.
#### Now let's send the values read using the driver to the Mosquitto MQTT Queue.
---

![IIOT Gateway MQTTBROKER](resources/images/IOTGW-DETAILS-HIGHLIGHT-MQTTBROKER.png)

#### To setup Mosquitto MQTT Broker, please visit the following link -
[Mosquitto MQTT Broker](https://mosquitto.org/download/)

- Follow the instructions specific to you OS. 
- After installation, start the Mosquitto service by running mosquitto - (service mosquitto restart - for linux)


- In a terminal(command prompt for windows), subscribe to the topic - arduino by running `mosquitto_sub -t arduino`
- Now let us see if we can publish our sensor data to the MQTT Broker by running the following code.

In [ ]:
import time
for i in range(100):    
    time.sleep(1)
    val=readAnalog()
    print(val,end="\t")
    !mosquitto_pub -t arduino -m {val}

#### You must now be able to see the values being published in the terminal.

In the next tutorial, we will see how to subscribe to the messages published to the MQTT Queue and then forward them to the IIOT Middleware. See you in the next tutorial.